In [1]:
import pandas as pd
import numpy as np

#### Layering: divide the data into N layers, make sure every layer has the same distribution of 0-1

## load 2017 train

In [2]:
df = pd.read_table('../data/water/txt/2017waterDataTraining.txt',delim_whitespace=True)

In [3]:
df.describe()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2
count,110812.000000,110814.000000,110815.000000,110815.000000,110815.000000,110815.000000,110815.000000,110815.000000,110815.000000
mean,4.567565,0.138432,8.369319,753.422092,220.801424,0.016335,0.097678,1463.273772,939.937698
std,0.455927,0.008487,0.214405,23.570613,18.309438,0.008463,0.011445,146.238918,102.770124
min,3.600000,0.000000,4.000000,300.000000,0.000000,0.000000,0.000000,1052.000000,479.000000
25%,4.100000,0.130000,8.290000,752.000000,212.000000,0.013000,0.091000,1362.000000,879.000000
50%,4.700000,0.140000,8.390000,758.000000,216.000000,0.016000,0.095000,1457.000000,942.000000
75%,4.900000,0.140000,8.460000,760.000000,235.000000,0.019000,0.103000,1555.000000,1001.000000
max,10.100000,0.181165,8.755965,894.000000,2500.000000,0.500000,1.000000,2070.000000,1248.000000


In [4]:
df = df.reset_index()
Time = np.zeros(df.shape[0]).astype("str")
for i in range(len(df)):
    Time[i] = df['index'][i]+" "+ df['Time'][i]
df['Time'] = Time
df = df.drop(['index'], axis=1)

## feature engineering


It looks like we have 14 columns to help us predict our classification. We will drop fnlwgt and education and then convert our categorical features to dummy variables. We will also convert our label to 0 and 1 where 1 means the person made more than $50k



In [5]:
drop_columns = ['Time']
continuous_features = ['Tp', 'Cl', 'pH', 'Redox', 'Leit', 'Trueb', 'Cl_2', 'Fm', 'Fm_2']
cat_features =[]

In [6]:
all_df_dummies = pd.get_dummies(df, columns=cat_features)

In [7]:
all_df_dummies.drop(drop_columns, 1, inplace=True)
# delte NA datas
all_df_dummies = all_df_dummies.dropna(axis=0)

In [8]:
X_train = all_df_dummies.drop(['EVENT'], axis=1) # Series
y_train = all_df_dummies['EVENT'].apply(lambda x: 0 if x == False else 1) # Series

In [9]:
train = pd.concat([X_train,y_train], axis=1)

In [10]:
train.head()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
0,4.4,0.14,8.38,755.0,232.0,0.009,0.110,1428.0,1020.0,0
1,4.4,0.14,8.38,755.0,232.0,0.009,0.111,1436.0,1018.0,0
2,4.4,0.14,8.38,755.0,232.0,0.014,0.113,1471.0,1019.0,0
3,4.4,0.14,8.37,755.0,232.0,0.015,0.111,1457.0,1015.0,0
4,4.4,0.14,8.38,755.0,232.0,0.013,0.111,1476.0,1019.0,0


In [11]:
train.describe()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
count,110812.000000,110812.000000,110812.000000,110812.000000,110812.000000,110812.000000,110812.000000,110812.000000,110812.000000,110812.000000
mean,4.567565,0.138432,8.369318,753.421914,220.801554,0.016335,0.097678,1463.275620,939.937218,0.015702
std,0.455927,0.008487,0.214408,23.570908,18.309669,0.008463,0.011446,146.240459,102.771474,0.124322
min,3.600000,0.000000,4.000000,300.000000,0.000000,0.000000,0.000000,1052.000000,479.000000,0.000000
25%,4.100000,0.130000,8.290000,752.000000,212.000000,0.013000,0.091000,1362.000000,879.000000,0.000000
50%,4.700000,0.140000,8.390000,758.000000,216.000000,0.016000,0.095000,1457.000000,942.000000,0.000000
75%,4.900000,0.140000,8.460000,760.000000,235.000000,0.019000,0.103000,1555.000000,1001.000000,0.000000
max,10.100000,0.181165,8.755965,894.000000,2500.000000,0.500000,1.000000,2070.000000,1248.000000,1.000000


In [10]:
train.to_csv('../data/water/csv/train2017.csv', encoding='utf-8', index=False)

#### layer sampling

In [11]:
# print("============ layer sampling ============")
# train_layer = Layering(train, kf)
# array = train_layer.values
# X_train = array[:, 0:-1] # ndarray
# y_train = array[:, -1] # ndarray

Train/Validation Split

In [12]:
# X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, y_train, test_size=valid_size,
#                                                 stratify = y_train, random_state = random_seed)
# # stratify： 按正负样本原始比例random_seed分配给train 和 valid

#### Do somte sampling on the train data to solve data imblance problem

In [13]:
# X_train_oversampled, y_train_oversampled = Smoter(X_tr, y_tr, is_random=True)
# print("============ SMOTE ============")
# print("train: %d, contains %.4f of 0 , after SMOTE: train: %d contains %.4f of 1" %(X_train.shape[0], (y_train == 0).sum()/y_train.shape[0], X_train_oversampled.shape[0], (y_train_oversampled == 0).sum()/y_train_oversampled.shape[0]))

### normalize the train and valid

fulfill the Na with median, then standardized the data, output type ndarray

In [14]:
# clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
#                            ('std_scaler', preprocessing.StandardScaler()),])
# X_train_oversampled = clean_pipeline.fit_transform(X_train_oversampled)
# X_vld = clean_pipeline.fit_transform(X_vld)

### transfer y into probability vector

In [15]:
# y_train_oversampled_pro = np.zeros([y_train_oversampled.shape[0], 2])
# for i in range(len(y_train_oversampled)):
#     if y_train_oversampled[i] == 1:
#         y_train_oversampled_pro[i] = np.array([0, 1])
#     else:
#         y_train_oversampled_pro[i] = np.array([1, 0])
# y_train_oversampled = y_train_oversampled_pro    

# y_vld_pro = np.zeros([y_vld.shape[0], 2])
# for i in range(len(y_vld)):
#     if y_vld[i] == 1:
#         y_vld_pro[i] = np.array([0, 1])
#     else:
#         y_vld_pro[i] = np.array([1, 0])
# y_vld = y_vld_pro

## load 2017 test

In [12]:
lines = open("../data/water/txt/2017waterDataTesting.txt").readlines()
num_lines = len(lines) - 1

X_test = np.ones((num_lines, 9))
y_test = np.ones((num_lines, 1))
flag = 0

lines = np.delete(lines, 0, axis = 0)
i = 0

for line in lines:
    data_line = line.split()
    feature = data_line[3:12]
    for k in range(9):
        if feature[k] == 'NA':
            flag = 1
            break
    if flag == 1:
        flag = 0
        continue    # jump out of the loop
    X_test[i] = feature    
    if data_line[12] == 'FALSE':
        y_test[i] = 0
    elif data_line[12] == 'TRUE':
        y_test[i] = 1
    i += 1


# X_test = clean_pipeline.fit_transform(X_test) 

test = np.concatenate([X_test, y_test], axis=1)

# y_test_pro = np.zeros([y_test.shape[0], 2])
# for i in range(len(y_test)):
#     if y_test[i] == 1:
#         y_test_pro[i] = np.array([0, 1])
#     else:
#         y_test_pro[i] = np.array([1, 0])
# y_test = y_test_pro

In [13]:
test = pd.DataFrame(test, columns =['Tp', 'Cl', 'pH', 'Redox', 'Leit', 'Trueb', 'Cl_2', 'Fm', 'Fm_2', 'EVENT'])
test.head()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
0,4.4,0.14,8.38,755.0,232.0,0.009,0.110,1428.0,1020.0,0.0
1,4.4,0.14,8.38,755.0,232.0,0.009,0.111,1436.0,1018.0,0.0
2,4.4,0.14,8.38,755.0,232.0,0.014,0.113,1471.0,1019.0,0.0
3,4.4,0.14,8.37,755.0,232.0,0.015,0.111,1457.0,1015.0,0.0
4,4.4,0.14,8.38,755.0,232.0,0.013,0.111,1476.0,1019.0,0.0


In [14]:
test.describe()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
count,244668.000000,244668.000000,244668.000000,244668.000000,244668.000000,244668.000000,244668.000000,244668.000000,244668.000000,244668.000000
mean,4.111409,0.349091,6.615673,575.860676,166.651494,0.253004,0.317009,1141.015212,717.466612,0.249113
std,1.876890,0.363771,3.140380,321.628357,93.762850,0.417314,0.381700,652.814631,413.745469,0.432500
min,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,3.800000,0.140000,8.180000,722.305130,203.000000,0.015000,0.095000,1175.000000,693.000000,0.000000
50%,4.700000,0.150000,8.330000,754.000000,211.000000,0.023000,0.106000,1423.000000,896.000000,0.000000
75%,5.200000,0.180000,8.420000,759.000000,219.000000,0.034000,0.132000,1557.000000,991.000000,0.000000
max,10.100000,1.000000,8.755965,894.000000,2500.000000,1.000000,1.000000,2093.000000,1284.000000,1.000000


In [18]:
test.to_csv('../data/water/csv/test2017.csv', encoding='utf-8', index=False)